In [8]:
import matplotlib.pyplot as plt
import numpy as np
import helper

import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import pandas as pd
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import math
from torcheval.metrics import BinaryAccuracy
import os
import torchmetrics
import torch.jit as jit
import timm
from typing import List
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size=4
image_count=25
img_size=256
tf = ToTensor()


In [2]:
train_image_No_closure_path='../../data/image_data/**/No_closure/*'
train_image_Oropharynx_path='../../data/image_data/**/Oropharynx/*'
train_image_Tonguebase_path='../../data/image_data/**/Tonguebase/*'
train_image_Epiglottis_path='../../data/image_data/**/Epiglottis/*'
test_image_No_closure_path='../../data/image_data_test/**/No_closure/*'
test_image_Oropharynx_path='../../data/image_data_test/**/Oropharynx/*'
test_image_Tonguebase_path='../../data/image_data_test/**/Tonguebase/*'
test_image_Epiglottis_path='../../data/image_data_test/**/Epiglottis/*'
# class CustomDataset(Dataset):
#     def __init__(self, image_list, label_list):
#         self.img_path = image_list

#         self.label = label_list


#     def __len__(self):
#         return len(self.label)

#     def __getitem__(self, idx):
#         image_tensor = torch.empty((image_count,3, img_size, img_size))
        
#         image_file_list = glob(self.img_path[idx]+'/*.jpg')
#         if len(image_file_list)>image_count:
#             image_index = torch.randint(low=0, high=len(
#                 image_file_list)-image_count, size=(1,))
#             count = 0
#             for index in range(image_index[0],image_index[0]+image_count):
#                 image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
#                 image_tensor[count] = image
#                 count += 1
#         else:
#             count = 0
#             for index in range(len(image_file_list)):
#                 image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
#                 image_tensor[count] = image
#                 count += 1
#             for i in range(image_count-count):
#                 image = 1-tf(Image.open(image_file_list[i]).resize((img_size,img_size)))
#                 image_tensor[count] = image
#                 count += 1
#         label_tensor =  self.label[idx]
#         return image_tensor, label_tensor
    
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list

        self.label = label_list


    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        image_tensor = self.img_path[idx]
    
        label_tensor =  self.label[idx]
        return image_tensor, label_tensor

train_image_list = []
train_label_list = []
image_No_closure_list = glob(train_image_No_closure_path)
image_No_closure_label = torch.zeros(len(image_No_closure_list), 1)
image_Oropharynx_list = glob(train_image_Oropharynx_path)
image_Oropharynx_label = torch.ones(len(image_Oropharynx_list), 1)
image_Tonguebase_list = glob(train_image_Tonguebase_path)
image_Tonguebase_label = torch.ones(len(image_Tonguebase_list), 1)*2
image_Epiglottis_list = glob(train_image_Epiglottis_path)
image_Epiglottis_label = torch.ones(len(image_Epiglottis_list), 1)*3

train_image_list.extend(image_No_closure_list)
train_image_list.extend(image_Oropharynx_list)
train_image_list.extend(image_Tonguebase_list)
train_image_list.extend(image_Epiglottis_list)
train_label_list.extend(image_No_closure_label)
train_label_list.extend(image_Oropharynx_label)
train_label_list.extend(image_Tonguebase_label)
train_label_list.extend(image_Epiglottis_label)

train_image_tensor = torch.empty((len(train_image_list),image_count,3, img_size, img_size))
for i in tqdm(range(len(train_image_list))):   
    image_file_list = glob(train_image_list[i]+'/*.jpg')
    if len(image_file_list)>image_count*2:
        image_index = torch.randint(low=0, high=len(
            image_file_list)-image_count*2, size=(1,))
        count = 0
        for index in range(image_index[0],image_index[0]+image_count*2,2):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            train_image_tensor[i,count] = image
            count += 1
    else:
        count = 0
        for index in range(0,len(image_file_list),2):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            train_image_tensor[i,count] = image
            count += 1
        for j in range(0,(image_count-count),2):
            image = 1-tf(Image.open(image_file_list[j]).resize((img_size,img_size)))
            train_image_tensor[i,count] = image
            count += 1

test_image_list = []
test_label_list = []
image_No_closure_list = glob(test_image_No_closure_path)
image_No_closure_label = torch.zeros(len(image_No_closure_list), 1)
image_Oropharynx_list = glob(test_image_Oropharynx_path)
image_Oropharynx_label = torch.ones(len(image_Oropharynx_list), 1)
image_Tonguebase_list = glob(test_image_Tonguebase_path)
image_Tonguebase_label = torch.ones(len(image_Tonguebase_list), 1)*2
image_Epiglottis_list = glob(test_image_Epiglottis_path)
image_Epiglottis_label = torch.ones(len(image_Epiglottis_list), 1)*3

test_image_list.extend(image_No_closure_list)
test_image_list.extend(image_Oropharynx_list)
test_image_list.extend(image_Tonguebase_list)
test_image_list.extend(image_Epiglottis_list)
test_label_list.extend(image_No_closure_label)
test_label_list.extend(image_Oropharynx_label)
test_label_list.extend(image_Tonguebase_label)
test_label_list.extend(image_Epiglottis_label)

test_image_tensor = torch.empty((len(test_image_list),image_count,3, img_size, img_size))
for i in tqdm(range(len(test_image_list))):   
    image_file_list = glob(test_image_list[i]+'/*.jpg')
    if len(image_file_list)>image_count:
        image_index = torch.randint(low=0, high=len(
            image_file_list)-image_count, size=(1,))
        count = 0
        for index in range(image_index[0],image_index[0]+image_count):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            test_image_tensor[i,count] = image
            count += 1
    else:
        count = 0
        for index in range(len(image_file_list)):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            test_image_tensor[i,count] = image
            count += 1
        for j in range(image_count-count):
            image = 1-tf(Image.open(image_file_list[j]).resize((img_size,img_size)))
            test_image_tensor[i,count] = image
            count += 1
            
train_dataset = CustomDataset(train_image_tensor, F.one_hot(torch.tensor(train_label_list).to(torch.int64)))

test_dataset = CustomDataset(test_image_tensor, F.one_hot(torch.tensor(test_label_list).to(torch.int64)))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

  0%|          | 0/1937 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
class TimeDistributed(nn.Module):
    def __init__(self, layer, time_steps, *args):        
        super(TimeDistributed, self).__init__()
        
        self.layers = nn.ModuleList([layer(*args) for i in range(time_steps)])

    def forward(self, x):
        batch_size, time_steps, C, H, W = x.size()
        output = torch.tensor([])
        for i in range(time_steps):
          output_t = self.layers[i](x[:, i, :, :, :])
          output_t  = y.unsqueeze(1)
          output = torch.cat((output, output_t ), 1)
        return output
    
class FeatureExtractor(nn.Module):
    """Feature extoractor block"""
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        cnn1= timm.create_model('efficientnet_b2', pretrained=True)
        self.feature_ex = nn.Sequential(*list(cnn1.children())[:-1])

    def forward(self, inputs):
        features = self.feature_ex(inputs)
        
        return features
    
class AttentionMILModel(nn.Module):
    def __init__(self, num_classes, image_feature_dim,feature_extractor_scale1: FeatureExtractor):
        super(AttentionMILModel, self).__init__()
        self.num_classes = num_classes
        self.image_feature_dim = image_feature_dim

        # Remove the classification head of the CNN model
        self.feature_extractor = feature_extractor_scale1
        
        self.dropout = nn.Dropout(0.5)

        self.dense1 = nn.Linear(256, 128)

        self.attention = nn.Sequential(
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        self.time_step=TimeDistributed(
    def forward(self, inputs):
        batch_size, num_tiles, channels, height, width = inputs.size()
        
        # Flatten the inputs
        inputs = inputs.view(-1, channels, height, width)
        
        # Feature extraction using the pre-trained CNN
        features = self.feature_extractor(inputs)  # Shape: (batch_size * num_tiles, 2048, 1, 1)
        
        # Reshape features
        features = features.view(batch_size, num_tiles, -1)  # Shape: (batch_size, num_tiles, 2048)
        features
        
        return features
Feature_Extractor=FeatureExtractor()
model = AttentionMILModel(4,1408,Feature_Extractor)
model = model.to(device)
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=4).to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-4)
summary(model,(batch_size,image_count,3,img_size,img_size))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [FeatureExtractor: 1, Sequential: 2, Conv2d: 3, BatchNormAct2d: 3, Identity: 4, SiLU: 4, Sequential: 3, Sequential: 4, DepthwiseSeparableConv: 5, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, SiLU: 7, SqueezeExcite: 6, Conv2d: 7, SiLU: 7, Conv2d: 7, Sigmoid: 7, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, Identity: 7, DepthwiseSeparableConv: 5, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, SiLU: 7, SqueezeExcite: 6, Conv2d: 7, SiLU: 7, Conv2d: 7, Sigmoid: 7, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, Identity: 7, Identity: 6, Sequential: 4, InvertedResidual: 5, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, SiLU: 7, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, SiLU: 7, SqueezeExcite: 6, Conv2d: 7, SiLU: 7, Conv2d: 7, Sigmoid: 7, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, Identity: 7, InvertedResidual: 5, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, SiLU: 7, Conv2d: 6, BatchNormAct2d: 6, Identity: 7, 

In [4]:
MIN_loss=5000
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
sig=nn.Sigmoid()
val_acc_list=[]
MIN_acc=0

numSample_list = [len(image_No_closure_list),len(image_Oropharynx_list),len(image_Tonguebase_list),len(image_Epiglottis_list)]
weights = torch.Tensor([1 - (x / sum(numSample_list)) for x in numSample_list]).to(device)
for epoch in range(1000):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    model.train()
    for x, y in train:
        
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = F.cross_entropy(predict.softmax(dim=1), y,weight=weights) # cost 구함
        acc=accuracy(predict.softmax(dim=1).argmax(dim=1),y.argmax(dim=1))
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{1000} Step: {count+1} loss : {running_loss/count:.4f} accuracy: {acc_loss/count:.4f}")
    train_loss_list.append((running_loss/count))
    train_acc_list.append((acc_loss/count).cpu().detach().numpy())
#validation
    val=tqdm(validation_dataloader)
    model.eval()
    count=0
    val_running_loss=0.0
    acc_loss=0
    with torch.no_grad():
        for x, y in val:
            y = y.to(device).float()
            count+=1
            x=x.to(device).float()
            predict = model(x).to(device)
            cost = F.cross_entropy(predict.softmax(dim=1), y,weight=weights) # cost 구함
            acc=accuracy(predict.softmax(dim=1).argmax(dim=1),y.argmax(dim=1))
            val_running_loss+=cost.item()
            acc_loss+=acc
            val.set_description(f"Validation epoch: {epoch+1}/{1000} Step: {count+1} loss : {val_running_loss/count:.4f}  accuracy: {acc_loss/count:.4f}")
        val_loss_list.append((val_running_loss/count))
        val_acc_list.append((acc_loss/count).cpu().detach().numpy())
    if epoch%100==5:
        plt.figure(figsize=(10,5))
        plt.subplot(1, 2, 1) 
        plt.title('loss_graph')
        plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
        plt.plot(np.arange(epoch+1),val_loss_list,label='validation_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.subplot(1, 2, 2)  
        plt.title('acc_graph')
        plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
        plt.plot(np.arange(epoch+1),val_acc_list,label='validation_acc')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.show()
        
        
    if MIN_loss>(val_running_loss/count):
        torch.save(model.state_dict(), '../../model/BiLSTM_MIL_eff_loss25_callback.pt')
        MIN_loss=(val_running_loss/count)
        
    if MIN_acc<(acc_loss/count):
        torch.save(model.state_dict(), '../../model/BiLSTM_MIL_eff_acc25_callback.pt')
        MIN_acc=(acc_loss/count)
        
torch.save(model.state_dict(), '../../model/BiLSTM_eff_50_MIL.pt')

  0%|          | 0/484 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/484 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

KeyboardInterrupt: 